In [2]:
import requests
import pprint
import json
import math
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### 법정동코드 불러오기

In [2]:
sigungu_bjdong = pd.read_excel('data/법정동코드.xls')

In [3]:
sigungu_bjdong

,법정동코드,법정동명
0,1100000000,서울특별시
1,1111000000,서울특별시 종로구
2,1111010100,서울특별시 종로구 청운동
3,1111010200,서울특별시 종로구 신교동
4,1111010300,서울특별시 종로구 궁정동
...,...,...
488,1174010600,서울특별시 강동구 둔촌동
489,1174010700,서울특별시 강동구 암사동
490,1174010800,서울특별시 강동구 성내동
491,1174010900,서울특별시 강동구 천호동


### 법정동에 해당하는 코드만 분리 (시/구 제외)

In [4]:
sb = []
for code in sigungu_bjdong['법정동코드']:
    if str(code)[5:] != '00000':
        sb.append(code)
print(sb)

[1111010100, 1111010200, 1111010300, 1111010400, 1111010500, 1111010600, 1111010700, 1111010800, 1111010900, 1111011000, 1111011100, 1111011200, 1111011300, 1111011400, 1111011500, 1111011600, 1111011700, 1111011800, 1111011900, 1111012000, 1111012100, 1111012200, 1111012300, 1111012400, 1111012500, 1111012600, 1111012700, 1111012800, 1111012900, 1111013000, 1111013100, 1111013200, 1111013300, 1111013400, 1111013500, 1111013600, 1111013700, 1111013800, 1111013900, 1111014000, 1111014100, 1111014200, 1111014300, 1111014400, 1111014500, 1111014600, 1111014700, 1111014800, 1111014900, 1111015000, 1111015100, 1111015200, 1111015300, 1111015400, 1111015500, 1111015600, 1111015700, 1111015800, 1111015900, 1111016000, 1111016100, 1111016200, 1111016300, 1111016400, 1111016500, 1111016600, 1111016700, 1111016800, 1111016900, 1111017000, 1111017100, 1111017200, 1111017300, 1111017400, 1111017500, 1111017600, 1111017700, 1111017800, 1111017900, 1111018000, 1111018100, 1111018200, 1111018300, 111

In [9]:
sb = [1159010600, 1159010700, 1159010800, 1159010900, 1162010100, 1162010200, 1162010300, 1165010100, 1165010200, 1165010300, 1165010400, 1165010600, 1165010700, 1165010800, 1165010900, 1165011000, 1165011100, 1168010100, 1168010300, 1168010400, 1168010500, 1168010600, 1168010700, 1168010800, 1168011000, 1168011100, 1168011200, 1168011300, 1168011400, 1168011500, 1168011800, 1171010100, 1171010200, 1171010300, 1171010400, 1171010500, 1171010600, 1171010700, 1171010800, 1171010900, 1171011100, 1171011200, 1171011300, 1171011400, 1174010100, 1174010200, 1174010300, 1174010500, 1174010600, 1174010700, 1174010800, 1174010900, 1174011000]

### OPEN API 데이터 불러오기

In [5]:
df = pd.DataFrame()

In [6]:
key = 'Xa9F%2BlohBnyap8Pj%2FHmyDdz%2FozVb5GFsC5z06uWip4HPt7n2yNTwF8UqP3Oq%2F%2Fx2M78iBDteaqxGceksQRSxdA%3D%3D'

In [7]:
empty = {}

In [10]:
for code in sb:
    sigungu = str(code)[:5]
    bjdong = str(code)[5:]
    
    url = 'http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?sigunguCd={0}&bjdongCd={1}&numOfRows=100&ServiceKey={2}&_type=json'.format(sigungu, bjdong, key)
    response = requests.get(url)
    dict = json.loads(response.text)
    total = dict["response"]["body"]["totalCount"]
    page = math.ceil(total/100)
    print(code, page)
    
    for pageno in range(page):
        url = 'http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?sigunguCd={0}&bjdongCd={1}&numOfRows=100&pageNo={2}&ServiceKey={3}&_type=json'.format(sigungu, bjdong, pageno+1, key)
        response = requests.get(url)
        try:
            dict = json.loads(response.text)     
            doc = dict["response"]["body"]["items"]["item"]
        except json.JSONDecodeError:
            print("Empty response")
            empty['{}'.format(code)] = pageno
        
        if type(doc) is list:
            df = pd.concat([df, pd.DataFrame(doc)], axis=0, ignore_index=True)
        else:
            df = pd.concat([df, pd.DataFrame(doc, index=[0])], axis=0, ignore_index=True)

        print(url)

1159010600 2
http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?sigunguCd=11590&bjdongCd=10600&numOfRows=100&pageNo=1&ServiceKey=Xa9F%2BlohBnyap8Pj%2FHmyDdz%2FozVb5GFsC5z06uWip4HPt7n2yNTwF8UqP3Oq%2F%2Fx2M78iBDteaqxGceksQRSxdA%3D%3D&_type=json
http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?sigunguCd=11590&bjdongCd=10600&numOfRows=100&pageNo=2&ServiceKey=Xa9F%2BlohBnyap8Pj%2FHmyDdz%2FozVb5GFsC5z06uWip4HPt7n2yNTwF8UqP3Oq%2F%2Fx2M78iBDteaqxGceksQRSxdA%3D%3D&_type=json
1159010700 62
http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?sigunguCd=11590&bjdongCd=10700&numOfRows=100&pageNo=1&ServiceKey=Xa9F%2BlohBnyap8Pj%2FHmyDdz%2FozVb5GFsC5z06uWip4HPt7n2yNTwF8UqP3Oq%2F%2Fx2M78iBDteaqxGceksQRSxdA%3D%3D&_type=json
http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo?sigunguCd=11590&bjdongCd=10700&numOfRows=100&pageNo=2&ServiceKey=Xa9F%2BlohBnyap8Pj%2FHmyDdz%2FozVb5GFsC5z06uWip4HPt7n2yNTwF8UqP3Oq%2F%2Fx2M78iBDteaqxGceksQRSxdA%3D%3D&_type=json
h

In [11]:
df

,archArea,atchBldArea,atchBldCnt,bcRat,bjdongCd,bldNm,block,bun,bylotCnt,crtnDay,...,splotNm,stcnsDay,strctCd,strctCdNm,totArea,totDongTotArea,ugrndFlrCnt,useAprDay,vlRat,vlRatEstmTotArea
0,0.0,40.95,1,0.0,10100,청운벽산빌리지,,0001,0,20170201,...,,,21,철근콘크리트구조,4663.05,4663.05,2,19881111,0.0,0.00
1,0.0,40.95,1,0.0,10100,청운벽산빌리지,,0001,0,20170201,...,,,21,철근콘크리트구조,4686.15,4686.15,2,19881111,0.0,0.00
2,0.0,40.95,1,0.0,10100,청운벽산빌리지,,0001,0,20170201,...,,,21,철근콘크리트구조,6572.10,6572.10,2,19881111,0.0,0.00
3,0.0,40.95,1,0.0,10100,청운벽산빌리지,,0001,0,20170201,...,,,21,철근콘크리트구조,5489.43,5489.43,2,19881111,0.0,0.00
4,0.0,40.95,1,0.0,10100,청운벽산빌리지,,0001,0,20170201,...,,,21,철근콘크리트구조,4772.55,4772.55,2,19881111,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592339,1222.92,0.00,0,59.95,11000,아이메디컬,근린생활용지,0000,1,20230706,...,고덕강일공공주택지구,20191108,21,철근콘크리트구조,9750.58,9750.58,2,20201106,296.72,6053.12
592340,539.8,0.00,0,59.98,11000,아이에스타워,근린생활용지,0000,0,20230714,...,고덕강일공공주택지구,20220404,21,철근콘크리트구조,3864.49,3864.49,2,20230707,297.06,2673.57
592341,539.75,0.00,0,59.97,11000,SB메디컬타워,근린생활용지,0000,0,20230906,...,고덕강일공공주택지구,20220225,21,철근콘크리트구조,4247.24,4247.24,2,20230522,299.86,2698.75
592342,543.63,0.00,0,59.74,11000,골드프라자,근린생활용지,0000,0,20230914,...,공공주택지구,20190904,21,철근콘크리트구조,4240.56,4240.56,2,20200807,297.03,2702.95


In [12]:
df.to_csv('건축물대장_표제부.csv', index=False)

In [13]:
empty

{'1121510100': 66,
 '1123010300': 30,
 '1129013300': 54,
 '1130510100': 30,
 '1132010700': 39,
 '1138010700': 19,
 '1147010200': 27,
 '1159010100': 14,
 '1168010500': 15,
 '1171010400': 17,
 '1174010800': 40}

In [14]:
df['mainPurpsCdNm'].value_counts()

mainPurpsCdNm
단독주택          293209
공동주택          140393
제2종근린생활시설      61445
제1종근린생활시설      58134
업무시설            9969
교육연구시설          8136
노유자시설           3267
종교시설            2938
공장              2396
숙박시설            1879
자동차관련시설         1854
창고시설            1592
문화및집회시설         1296
판매시설            1224
근린생활시설           998
의료시설             801
위험물저장및처리시설       678
운동시설             339
교정및군사시설          290
운수시설             260
교육연구및복지시설        186
위락시설             173
동물및식물관련시설        164
관광휴게시설           155
분뇨.쓰레기처리시설       143
방송통신시설            85
수련시설              73
자원순환관련시설          72
묘지관련시설            46
판매및영업시설           43
                  37
발전시설              21
국방,군사시설           16
공공용시설             14
야영장시설              9
장례시설               6
다가구주택              2
가설건축물              1
Name: count, dtype: int64

In [15]:
df2 = df[df['mainPurpsCdNm'] == '업무시설']

In [20]:
df2

,archArea,atchBldArea,atchBldCnt,bcRat,bjdongCd,bldNm,block,bun,bylotCnt,crtnDay,...,splotNm,stcnsDay,strctCd,strctCdNm,totArea,totDongTotArea,ugrndFlrCnt,useAprDay,vlRat,vlRatEstmTotArea
558,0.0,331.11,3,0.0,10300,,,0001,0,20230904,...,,,21,철근콘크리트구조,7204.8200,7204.8200,1,,0.0,0.0000
601,0.0,0.00,0,0.0,10300,,,0008,0,20171205,...,,,31,일반철골구조,337.7200,337.7200,0,,32.1,337.7200
602,120.0,0.00,0,11.63,10300,,,0008,0,20200427,...,,20110712,31,일반철골구조,0.0000,0.0000,0,20111009,0.0,0.0000
756,0.0,0.00,0,0.0,10400,,,0077,7,20171118,...,,,21,철근콘크리트구조,6679.9500,6679.9500,3,19910430,0.0,0.0000
894,3.84,0.00,0,0.03,10500,주유사무실,,0067,32,20200427,...,,19950418,32,경량철골구조,3.8400,3.8400,0,19970630,0.03,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592142,211.14,0.00,0,59.81,11000,트리피움오피스텔,,0677,0,20190603,...,,20141015,21,철근콘크리트구조,2504.6250,2504.6300,2,20151113,556.05,1962.8650
592143,210.48,0.00,0,59.97,11000,유러피안하우스I,,0677,0,20230629,...,,20120420,21,철근콘크리트구조,2214.0700,2214.0700,2,20130524,495.09,1737.7800
592144,230.13,0.00,0,59.93,11000,유러피안하우스II,,0677,0,20190603,...,,20120920,21,철근콘크리트구조,2378.0700,2378.0700,2,20130830,481.66,1849.5600
592147,477.95,0.00,0,59.74,11000,강일큐브,,0679,0,20230629,...,,20110520,21,철근콘크리트구조,6988.1600,6988.1600,4,20120926,566.67,4533.3800


In [4]:
file = pd.read_csv('data/건축물대장_표제부.csv', encoding='utf-8-sig')

C:\Users\Puser\AppData\Local\Temp\ipykernel_15260\995148056.py:1: DtypeWarning: Columns (33,35,42,63,70) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('data/건축물대장_표제부.csv', encoding='utf-8-sig')


In [14]:
file[file['bldNm'] == '안녕인사동']

,archArea,atchBldArea,atchBldCnt,bcRat,bjdongCd,bldNm,block,bun,bylotCnt,crtnDay,...,splotNm,stcnsDay,strctCd,strctCdNm,totArea,totDongTotArea,ugrndFlrCnt,useAprDay,vlRat,vlRatEstmTotArea
4864,3447.88,0.0,0,58.91,12800,안녕인사동,,155,4,20230519,...,,20170522,42,철골철근콘크리트구조,45284.0,45284.0,4,20190808,473.58,27719.56


In [19]:
df.isnull().sum()

archArea            0
atchBldArea         0
atchBldCnt          0
bcRat               0
bjdongCd            0
                   ..
totDongTotArea      0
ugrndFlrCnt         0
useAprDay           0
vlRat               0
vlRatEstmTotArea    0
Length: 78, dtype: int64